# Libraries

In [1]:
import pandas as pd
import numpy as np
from binance.client import Client
import websocket
import json
import threading
import time
from datetime import datetime

# 1. Extract

Websocket

In [2]:
# Shared state
price_websocket = None

def on_message(ws, message):
    global price_websocket
    data = json.loads(message)
    wdata = data['k']
    price = float(wdata['c'])
    price_websocket = price

def on_open(ws):
    payload = {"method": "SUBSCRIBE",
            "params": ["btcusdt@kline_1s"],
               "id": 1}
    ws.send(json.dumps(payload))
def on_error(ws, error):
    print("Error:", error)

def on_close(ws, close_status, close_message):
    print("WebSocket closed")

def get_websocket_data():
    ws = websocket.WebSocketApp("wss://stream.binance.com:9443/ws",
                                on_open=on_open,
                                on_message=on_message,
                                on_error=on_error,
                                on_close=on_close )
    ws.run_forever()


thread = threading.Thread(target=get_websocket_data, daemon = True)
thread.start()

main_df = pd.DataFrame(columns=["timestamp", "price_websocket"])
for i in range(100):
    timestamp = datetime.utcnow().replace(microsecond=0)
    main_df = pd.concat([main_df, pd.DataFrame([{"timestamp": timestamp, "price_websocket": price_websocket}])], ignore_index=True)
    time.sleep(1)

Error: 'k'


API

In [3]:
api_key = "S0su5HSh9NeSfo9hD0r8LWSoxqX2LvgNa2QmlonmkQblmOonsktOogczff6SDi01"
api_secret = "bWrYTb4bVYIA8AVSTjyeZzOOzRMjnX2L9c5PwrHD3am1639QMpLBk9klU9pULA6x"

client = Client(api_key, api_secret)

def get_historical_klines(symbol, interval, lookback):
    try:
        klines = client.get_historical_klines(symbol, interval, lookback)
        raw_df = pd.DataFrame(klines, columns = [
            "timestamp", "open", "high", "low", "close", "volume",
            "close_time", "quote_asset_volume", "number_of_trades", "taker_buy_base_asset_volume",
            "taker_buy_quote_asset_volume", "ignore"
        ])
        raw_df = raw_df[["timestamp", "close"]]
        raw_df["timestamp"] = pd.to_datetime(raw_df["timestamp"], unit = "ms")
        raw_df.set_index("timestamp", inplace = True)
        raw_df = raw_df.astype(float)
        return raw_df
    except Exception as e:
        raise Exception(f"Error")

symbol = "BTCUSDT"
interval = "1s"
lookback = "100 sec ago UTC"

raw_df = get_historical_klines(symbol, interval, lookback)
raw_df.columns = ['price_rest']

In [4]:
btc_df = raw_df.merge(main_df, on='timestamp')
btc_df

,timestamp,price_rest,price_websocket
0,2025-05-02 19:49:13,96788.01,96788.01
1,2025-05-02 19:49:14,96788.00,96788.01
2,2025-05-02 19:49:15,96788.00,96788.00
3,2025-05-02 19:49:16,96788.00,96788.00
4,2025-05-02 19:49:17,96788.00,96788.00
...,...,...,...
93,2025-05-02 19:50:47,96805.00,96805.00
94,2025-05-02 19:50:48,96805.01,96805.00
95,2025-05-02 19:50:49,96805.00,96805.01
96,2025-05-02 19:50:50,96805.01,96805.00
